In [2]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import lxml

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd

from datetime import datetime

#Отображение колонок и строк в VScode
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
profs_all = pd.read_csv('profs.txt', header=None, names=['profs_names'])
profs_test = pd.read_csv('profs_test.txt', header=None, names=['profs_names'])

In [4]:
sber_job = pd.DataFrame(columns=['link', 'name', 'location', 'company', 'date', 'description', 'date_of_download'])
df_test = pd.DataFrame(columns=['link', 'name', 'location', 'company', 'date', 'description', 'date_of_download'])

In [5]:
df_test = pd.read_csv("all_sber.txt", sep=';')
df_test[:5]

,link,name,location,company,date,description,date_of_download
0,https://rabota.sber.ru/search/4190759,Главный сетевой инженер,г Москва,NaN,09 октября 2023,NaN,2023-10-09 16:50:42.989057
1,https://rabota.sber.ru/search/4206802,Инженер SRE/DevOps (Kubernetes),г Москва,NaN,09 октября 2023,NaN,2023-10-09 16:50:43.037504
2,https://rabota.sber.ru/search/4208406,Кредитный инспектор в отдел сложноструктуриров...,г Санкт-Петербург,NaN,09 октября 2023,NaN,2023-10-09 16:50:43.084766
3,https://rabota.sber.ru/search/4179283,Операционный менеджер ВИП офиса Сбер,"г Уфа, республика Башкортостан",NaN,09 октября 2023,NaN,2023-10-09 16:50:43.142268
4,https://rabota.sber.ru/search/4207926,Водитель инкассации,"г Муром, Владимирская область",NaN,09 октября 2023,NaN,2023-10-09 16:50:43.189496


In [12]:
class SberJobParser:

    url = 'https://rabota.sber.ru/search/'
    
    def __init__(self, vac_type):
        '''
        Для инциалиазации задается название вакансии vac_type
        '''
        self.vac_type = vac_type
        self.browser = webdriver.Chrome()
        self.browser.get(self.url)
        self.browser.maximize_window()
        self.browser.delete_all_cookies()
        time.sleep(2)

    def scroll_down_page(self, page_height=0):
        self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_page_height = self.browser.execute_script('return document.body.scrollHeight')
        if new_page_height > page_height:
            self.scroll_down_page(new_page_height)
        
    def stop(self):
        '''
        Выйти из Webdriver selenium
        '''
        self.browser.quit()
    
    def find_vacancies(self, profs):
        '''
        Функция для ввода названия вакансии (vac_type) в поисковую строку и клика клавиши Enter
        '''
        for i in range(0, len(profs)):

            top = self.browser.find_element(By.XPATH, '/html/body/div/div/div[2]/div[3]/div/div/div[1]')
            self.browser.execute_script("return arguments[0].scrollIntoView(true);", top)

            input_button = self.browser.find_element(By.XPATH, '/html/body/div/div/div[2]/div[3]/div/div/div[2]/div/div/div/div/input')

            input_button.send_keys(f"{profs['profs_names'][i]}")
            click_button = self.browser.find_element(By.CLASS_NAME, 'iEybKe ')
            click_button.click()
            time.sleep(5)

            # Прокрутка вниз до конца страницы
            self.scroll_down_page()

            try:
                vacs_bar = self.browser.find_element(By.XPATH, '/html/body/div/div/div[2]/div[3]/div/div/div[3]/div/div[3]/div[2]')
                vacs = vacs_bar.find_elements(By.CLASS_NAME, 'fmUtEX')
                vacs = [div for div in vacs if 'fmUtEX' in str(div.get_attribute('class'))]
            
                for vac in vacs:
                    vac_info = {}
                    vac_info['link'] = vac.find_element(By.TAG_NAME, 'a').get_attribute('href')
                    vac_info['name'] = vac.find_element(By.CLASS_NAME, 'iUPRtQ').text
                    vac_info['location'] = vac.find_element(By.CLASS_NAME, 'kkMmnR').text
                    #vac_info['company'] = vac.find_element(By.CLASS_NAME, 'result-item-unit').text
                    vac_info['date'] = vac.find_element(By.CLASS_NAME, 'cMyOwV').text
                    vac_info['date_of_download'] = datetime.now()
                    df_test.loc[len(df_test)] = vac_info
                input_button.clear()
            except:
                input_button.clear()
                pass
            
        df_test = df_test.drop_duplicates(subset=['link'], keep='last')

    def find_description(self):
        for descr in range(len(df_test)):
            try:
                link = df_test.loc[descr, 'link']
                self.browser.get(link)
                self.browser.delete_all_cookies()
                time.sleep(3)

                df_test.loc[descr, 'description'] = self.browser.find_element(By.CLASS_NAME, 'eBiZdk').text
            except:
                pass

    def save_df(self):
        df_test.to_csv(f"all_sber.txt", index=False, sep=';')

In [26]:
parser = SberJobParser(profs_all)
parser.find_vacancies(profs_all)
parser.stop()

In [13]:
parser = SberJobParser(profs_all)
parser.find_description()
parser.stop()

In [15]:
df_test = df_test.drop_duplicates(subset=['link'], keep='first')

In [16]:
df_test[:12]

,link,name,location,company,date,description,date_of_download
0,https://rabota.sber.ru/search/4190759,Главный сетевой инженер,г Москва,NaN,09 октября 2023,SberInfra ищет в свою дружную команду Главного...,2023-10-09 16:50:42.989057
1,https://rabota.sber.ru/search/4206802,Инженер SRE/DevOps (Kubernetes),г Москва,NaN,09 октября 2023,Облачная платформа SberInfra.Cloud является фу...,2023-10-09 16:50:43.037504
2,https://rabota.sber.ru/search/4208406,Кредитный инспектор в отдел сложноструктуриров...,г Санкт-Петербург,NaN,09 октября 2023,Команда блока «корпоративно – инвестиционный б...,2023-10-09 16:50:43.084766
3,https://rabota.sber.ru/search/4179283,Операционный менеджер ВИП офиса Сбер,"г Уфа, республика Башкортостан",NaN,09 октября 2023,СберПервый – это офис премиального обслуживани...,2023-10-09 16:50:43.142268
4,https://rabota.sber.ru/search/4207926,Водитель инкассации,"г Муром, Владимирская область",NaN,09 октября 2023,Сбер — лучший работодатель России. У нас 14 ты...,2023-10-09 16:50:43.189496
5,https://rabota.sber.ru/search/4206833,Водитель инкассации,"г Ижевск, республика Удмуртская",NaN,09 октября 2023,Сбер — лучший работодатель России. У нас 14 ты...,2023-10-09 16:50:43.244889
6,https://rabota.sber.ru/search/4208983,Водитель инкассации,"г Набережные Челны, республика Татарстан",NaN,09 октября 2023,Сбер — лучший работодатель России. У нас 14 ты...,2023-10-09 16:50:43.289936
7,https://rabota.sber.ru/search/4212886,Клиентский менеджер офиса Домклик,"г. Мурино, Ленинградская область",NaN,09 октября 2023,Наша команда занимается ипотечным кредитование...,2023-10-09 16:50:43.336433
8,https://rabota.sber.ru/search/4174354,Главный специалист технической поддержки,"г Тула, Тульская область",NaN,09 октября 2023,Сбер — лучший работодатель России. У нас и 14 ...,2023-10-09 16:50:43.384001
9,https://rabota.sber.ru/search/4153779,Ведущий экономист в службу финансового менеджм...,"г Нижний Новгород, Нижегородская область",NaN,09 октября 2023,Отдел экономической оценки ищет специалиста в ...,2023-10-09 16:50:43.439943


In [33]:
df_test.to_csv(f"all_sber.txt", index=False, sep='|')